# Corrélation entre le taux de ponctualité fourni par la SNCF et les Tweets

In [ ]:
import tweepy
import re as regex
import pandas as pd
from datetime import datetime

### Définir la ligne de transilien à analyser

In [ ]:
transilien_name = 'rer B'

### Définir la période pendant laquelle les données seront analysées

In [ ]:
start_time = datetime(2022, 10, 1, 00, 00, 00)
end_time = datetime(2022, 11, 1, 00, 00, 00)

print(f"{start_time=}")
print(f"{end_time=}")

## Extraire le taux de ponctualité des données SNCF

In [ ]:
ponctualite_mensuelle_transilien_df = pd.read_csv("files/ponctualite-mensuelle-transilien.csv", sep=';')
ponctualite_mensuelle_transilien_df.head()

In [ ]:
taux_ponctualite_transilien = ponctualite_mensuelle_transilien_df[ponctualite_mensuelle_transilien_df["Nom de la ligne"].str.lower() == transilien_name.lower()]
taux_ponctualite_transilien

In [ ]:
filter_by_month_df = taux_ponctualite_transilien[taux_ponctualite_transilien.Date == f'{start_time.year}-{start_time.month}']

filter_by_month_df

In [ ]:
if filter_by_month_df.empty:
    print("WARNING: you should shows an other date")

In [ ]:
filter_by_month_df["Taux de ponctualité"]

## Collecter et analyser les données Twitter

### Connection au [site développeur de Twitter](https://developer.twitter.com/en) afin d'obtenir des tokens

In [ ]:
TWITTER_BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAOkWlQEAAAAAf8qKdgherDuI1drlvoJIYjF1wsU%3DVtR78wTC9oFMEPbYp3ArbhH3rBe3vk21OwdtvOWrtnBixoA7Wh'
TWITTER_CONSUMER_KEY = 'JanEaDdu5pa4WjgIugVJ8YZV5'
TWITTER_CONSUMER_SECRET = 'it0MnrfuCrZF0ydsPfjSlMPJx2VDlJRcLslL1TND6LdAsV2sBb'

In [ ]:
# define tweepy client
client = tweepy.Client(bearer_token=TWITTER_BEARER_TOKEN,
                       consumer_key=TWITTER_CONSUMER_KEY,
                       consumer_secret=TWITTER_CONSUMER_SECRET,
                       wait_on_rate_limit=True)

In [ ]:
transilien_username = transilien_name.replace(' ', '')

transilien_information = client.get_user(username=transilien_username)
print(transilien_information.data.id)

In [ ]:
start_time = datetime(2022, 10, 1, 00, 00, 00)
end_time = datetime(2023, 1, 1, 00, 00, 00)

print(f"{start_time=}")
print(f"{end_time=}")

In [ ]:
transilien_id = transilien_information.data.id

# get all tweets of a user during a period
tweets = tweepy.Paginator(
        client.get_users_tweets,
        id=transilien_id,
        tweet_fields=[
            'id',
            'text',
            'author_id',
            'conversation_id',
            'created_at',
            'lang',
            'referenced_tweets',
            'public_metrics',
        ],
        start_time=start_time,
        end_time=end_time,
        exclude="replies",
    ).flatten(limit=1000)

### Trouver une méthode pour compter le nombre d'incidents

In [ ]:
number_incidents = 0

for tweet in tweets:
    regex_result = regex.search("(Fin\sd['’])|(L')(incident)(\sest\sterminé)", tweet.text)
    if regex_result is not None:
        number_incidents += 1

print(number_incidents)

## Interpréter les données GTFS afin d'obtenir le nombre de trains impactés par les perturbations

### Charger les données depuis les fichiers gtfs

In [ ]:
stop_times = pd.read_csv('./files/transilien-gtfs/stop_times.txt', sep=",")
stop_times.head()

In [ ]:
stops = pd.read_csv('./files/transilien-gtfs/stops.txt', sep=",")
stops.head()

In [ ]:
trips=pd.read_csv('./files/transilien-gtfs/trips.txt', sep=",")
trips.head()

In [ ]:
routes = pd.read_csv('./files/transilien-gtfs/routes.txt', sep=",")
routes.head()

### Merger plusieurs Dataframe pour obtenir les données finales

In [ ]:
trips_stops = pd.merge(stop_times, trips, on='trip_id', how='outer')
trips_stops.head()

In [ ]:
routes_trips_stops = pd.merge(routes, trips_stops, on='route_id', how='outer')
routes_trips_stops.head()

### Filtrer le Dataframe par le transilien que nous analysons

In [ ]:
transilien_shortname = transilien_name.split(' ')[1]

only_transilien_df = routes_trips_stops[routes_trips_stops["route_short_name"] == transilien_shortname]
only_transilien_df

### Filtrer les données pour ne garder que les trains

In [ ]:
def find_routes_types(df, filter=None):
    filter_df = None

    display(df.groupby(["route_type"])["route_type"].count())
    if filter is not None:
        filter_df = df[df["route_type"] == filter]
        display(filter_df)

    return filter_df

In [ ]:
only_transilien_df.groupby(["route_type"])["route_type"].count()
filter_df = find_routes_types(only_transilien_df, 2)

### Simplifier la donnée en ne gardant que ce qui est réellement intéressant

In [ ]:
simple_df = filter_df[["stop_id","trip_id", "arrival_time", "departure_time", "stop_sequence", "pickup_type", "drop_off_type", "direction_id"]]

### Filtrer pour ne garder que le premier train d'une séquence et obtenir le nombre total de trains sur une journée

In [ ]:
only_departure_stop = simple_df[filter_df["stop_sequence"].isin([1])]
only_departure_stop["trip_id"].unique()